In [1]:
import json
import torch
from torch import nn
import torch.nn.functional as F
from transformers import BertTokenizer
from transformers import BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = 'I will watch Memento tonight'
bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
                       truncation=True, return_tensors="pt")

print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

C:\Users\Admin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


tensor([[  101,   146,  1209,  2824,  2508, 26173,  3568,   102,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [2]:
labels = {'General':0,
          'Food':1,
          'Transportation':2,
          'Entertainment':3,
          'Sports':4
          }

In [3]:
import pandas as pd
import numpy as np
train_data = pd.read_csv('complete_train.csv')
test_data = pd.read_csv('complete_test.csv')

In [9]:
train_data['category'].value_counts()

General           422
Food              180
Transportation     21
Entertainment       2
Name: category, dtype: int64

In [14]:
train_general = train_data[train_data['category'] == "General"]

In [15]:
train_data.drop(train_data[train_data.category == "General"].index, inplace=True)

In [20]:
train_data = train_data.append(train_general[0:200], ignore_index=True)

In [21]:
# Drop null value
train_data = train_data.dropna()
test_data = test_data.dropna()

In [22]:
labels_train = [labels[label] for label in train_data['category']]
texts_train = [tokenizer(text,padding='max_length', max_length = 10, 
                       truncation=True, return_tensors="pt") for text in train_data['text']]
labels_test = [labels[label] for label in test_data['category']]
texts_test = [tokenizer(text,padding='max_length', max_length = 10, 
                       truncation=True, return_tensors="pt") for text in test_data['text']]

In [23]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(768, 200)
        self.linear = nn.Linear(200, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        lstm_output = self.lstm(dropout_output)
        linear_output = self.linear(lstm_output[0])
        final_layer = self.relu(linear_output)

        return final_layer

In [24]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, texts, labels):

        self.labels = labels
        self.texts = texts

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [25]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, learning_rate, epochs):

    train = Dataset(texts_train, labels_train)
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    test = Dataset(texts_test, labels_test)
    val_dataloader = torch.utils.data.DataLoader(test, batch_size=2, shuffle=True)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(test_data): .3f} \
                | Val Accuracy: {total_acc_val / len(test_data): .3f}')
                  
EPOCHS = 6
model = BertClassifier()
LR = 1e-6
              
train(model, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|                                                                                          | 0/185 [00:00<?, ?it/s]

Epochs: 1 | Train Loss:  0.811                 | Train Accuracy:  0.149                 | Val Loss:  0.802                 | Val Accuracy:  0.375


  0%|                                                                                          | 0/185 [00:00<?, ?it/s]

Epochs: 2 | Train Loss:  0.793                 | Train Accuracy:  0.384                 | Val Loss:  0.783                 | Val Accuracy:  0.463


  0%|                                                                                          | 0/185 [00:00<?, ?it/s]

...

Epochs: 4 | Train Loss:  0.726                 | Train Accuracy:  0.484                 | Val Loss:  0.703                 | Val Accuracy:  0.476


  0%|                                                                                          | 0/185 [00:00<?, ?it/s]

Epochs: 5 | Train Loss:  0.693                 | Train Accuracy:  0.511                 | Val Loss:  0.672                 | Val Accuracy:  0.524


100%|████████████████████████████████████████████████████████████████████████████████| 185/185 [03:07<00:00,  1.02s/it]


Epochs: 6 | Train Loss:  0.660                 | Train Accuracy:  0.595                 | Val Loss:  0.644                 | Val Accuracy:  0.552


In [15]:
model.eval()

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [32]:
torch.save(model.state_dict(), 'saved_model/tor.pt')

In [30]:
x = tokenizer("restaurant noodle ", padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

In [31]:
model(x.input_ids, x.attention_mask)

tensor([[0.3799, 0.5864, 0.0000, 0.0000, 0.0082]], grad_fn=<ReluBackward0>)

In [3]:
model = BertClassifier()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import torch
model.load_state_dict(torch.load('saved_model/tor.pt'))
model.eval()

BertClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru